# NT@B Software Workshop 1
### Data Acquisition & Cleaning
#### Saarang Panchavati and Abhinav Pottabathula

***

## Intro to Jupyter 
Welcome to Jupyter Notebook! Let's get you familiarized with Jupyter. If you're already familiar with the Jupyter environment you can move on to the next section.

Jupyter is really cool because you can execute your code in _cells_ instead of all at once. This is extremely useful for data science because you can iterate through different methods of analysis without having to run all your code at once. 

By default, you are in *command* mode in command mode, you can do the following:
1. Change a cells _type_: (code, markdown or raw). The type of this cell is markdown, where you can write stuff and make it look pretty
2. Add and Delete Cells
3. Run Cells

To re-enter *command* mode, just press `esc` on your keyboard.

Here are the most useful keyboard shortcuts:
1. `shift + enter` runs a cell
2. `a`,`b` adds a cell above and below the current one, respectively
3. `y` changes a cell to code type
4. `m` changes as cell to markdown type
5. `d+d` deletes the selected cell
6. `z+z` is undo
7. `i+i` interrupts the notebook (in case you have an infinite loop or something)

 

Some things to remember:
1. The number next to a cell is the order in which it was run, if there's a star there then that means it is still running. 
2. Jupyter saves variables even if you delete the cell, so if you want to re-run your notebook or run a cell, make sure you're aware of what the variables are set to!
3. Be sure to stay organized in your notebook — it can become very hard to find cells or portions of your code if you don't organize your code in a nice way

# convert this cell in to code type and run it with shift enter!
print("Hello World!"### Convert this cell in to code type and run it with shift enter!

In [ ]:
#interrupt with ii, interrupt quickly or this might break your laptop lol
while True:
    continue

***

## Intro to Pandas! 

__Pandas is THE library for datascience! It is really important to be familiar with it and understand how to use it. Let's dive in!__

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import zipfile
from pathlib import Path
%matplotlib inline


def fetch_and_cache(data_url, file, data_dir="data", force=False):
    """
    Download and cache a url and return the file object.
    
    data_url: the web address to download
    file: the file in which to save the results.
    data_dir: (default="data") the location to save the data
    force: if true the file is always re-downloaded 
    
    return: The pathlib.Path to the file.
    """
    data_dir = Path(data_dir)
    data_dir.mkdir(exist_ok=True)
    file_path = data_dir/Path(file)
    if force and file_path.exists():
        file_path.unlink()
    if force or not file_path.exists():
        print('Downloading...', end=' ')
        resp = requests.get(data_url)
        with file_path.open('wb') as f:
            f.write(resp.content)
        print('Done!')
    else:
        import time 
        created = time.ctime(file_path.stat().st_ctime)
        print("Using cached version downloaded at", created)
    return file_path

data_url = 'https://www.ssa.gov/oact/babynames/state/namesbystate.zip'
namesbystate_path = fetch_and_cache(data_url, 'namesbystate.zip')
zf = zipfile.ZipFile(namesbystate_path, 'r')

column_labels = ['State', 'Sex', 'Year', 'Name', 'Count']

def load_dataframe_from_zip(zf, f):
    with zf.open(f) as fh: 
        return pd.read_csv(fh, header=None, names=column_labels)

states = [
    load_dataframe_from_zip(zf, f)
    for f in sorted(zf.filelist, key=lambda x:x.filename) 
    if f.filename.endswith('.TXT')
]

baby_names = states[0]
for state_df in states[1:]:
    baby_names = pd.concat([baby_names, state_df])
baby_names = baby_names.reset_index().iloc[:, 1:]
ca = baby_names[baby_names['State'] == 'CA']

Downloading... 

KeyboardInterrupt: 

## Data Aggregration (Grouping Data Frames)

### Question 1a
To count the number of instances of each unique value in a `Series`, we can use the `value_counts()` [method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html) as `df["col_name"].value_counts()`. Count the number of different names for each Year in `CA` (California).  (You may use the `ca` DataFrame created above.)

**Note:** *We are not computing the number of babies but instead the number of names (rows in the table) for each year.*

<!--
BEGIN QUESTION
name: q1a
-->

In [ ]:
num_of_names_per_year = ...
num_of_names_per_year[:5]

### Question 1b
Count the number of different names for each gender in `CA`.


<!--
BEGIN QUESTION
name: q1b
-->

In [ ]:
num_of_names_per_gender = ...
num_of_names_per_gender

### Question 2: Groupby ###

Before we jump into using the [`groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) function in Pandas, let's recap how grouping works in general for tabular data through a guided set of questions based on a small toy dataset of movies and genres. 

**Note:** If you want to see a visual of how grouping of data works, here is a link to an animation [Groupby Animation](http://www.ds100.org/sp18/assets/lectures/lec03/03-groupby_and_pivot.pdf)

**Problem Setting:** This summer 2018, there were a lot of good and bad movies that came out. Below is a dataframe with 5 columns: name of the movie as a `string`, the genre of the movie as a `string`, the first name of the director of the movie as a `string`, the average rating out of 10 on Rotten Tomatoes as an `integer`, and the total gross revenue made by the movie as an `integer`. The point of these guided questions (parts a and b) below is to understand how grouping of data works in general, **not** how grouping works in code. We will worry about how grouping works in Pandas in 7c, which will follow.

Below is the `movies` dataframe we are using, imported from the `movies.csv` file located in the `lab02` directory.

In [ ]:
movies = pd.read_csv("movies.csv")
movies

### Question 2a

If we grouped the `movies` dataframe above by `genre`, how many groups would be in the output and what would be the groups? Assign `num_groups` to the number of groups created (hard-code) and fill in `genre_list` as a list containing the names of genres as strings that represent the groups.

<!--
BEGIN QUESTION
name: q2a
-->

In [ ]:
num_groups = ...
genre_list = ...

### Question 2b

Whenever we group tabular data, it is usually the case that we need to aggregate values from the ungrouped column(s). If we were to group the `movies` dataframe above by `genre`, which column(s) in the `movies` dataframe would it make sense to aggregate if we were interested in finding how well each genre did in the eyes of people? Fill in `agg_cols` with the column name(s).

<!--
BEGIN QUESTION
name: q2b
-->

In [ ]:
agg_cols = ...

Now, let's see groupby in action, instead of keeping everything abstract. To aggregate data in Pandas, we use the .groupby() function. The code below will group the movies dataframe by genre and find the average revenue and rating for each genre. You can verify you had the same number of groups as what you answered in 7a.

In [ ]:
movies.loc[:, ['genre', 'rating', 'revenue']].groupby('genre').mean()

### Question 2c

Let's move back to baby names and specifically, the `ca` dataframe. Find the sum of `Count` for each `Name` in the `ca` table. You should use `df.groupby("col_name").sum()`. Your result should be a Pandas Series.

**Note:** *In this question we are now computing the number of registered babies with a given name.*

<!--
BEGIN QUESTION
name: q2c
-->

In [ ]:
count_for_names = ...
count_for_names.sort_values(ascending=False)[:5]

### Question 2d

Find the sum of `Count` for each female name after year 1999 (`>1999`) in California.


<!--
BEGIN QUESTION
name: q2d
-->

In [ ]:
female_name_count = ...
female_name_count.sort_values(ascending=False)[:5]

### Question 3: Grouping Multiple Columns

Let's move back to the `movies` dataframe. Which of the following lines of code will output the following dataframe? Write your answer (hard-coded) as either 1, 2, 3, or 4. Recall that the arguments to `pd.pivot_table` are as follows: `data` is the input dataframe, `index` includes the values we use as rows, `columns` are the columns of the pivot table, `values` are the values in the pivot table, and `aggfunc` is the aggregation function that we use to aggregate `values`.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>rating</th>
      <th>5</th>
      <th>6</th>
      <th>7</th>
      <th>8</th>
    </tr>
    <tr>
      <th>genre</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Action &amp; Adventure</th>
      <td>208681866.0</td>
      <td>129228350.0</td>
      <td>318344544.0</td>
      <td>6708147.0</td>
    </tr>
    <tr>
      <th>Animation</th>
      <td>374408165.0</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Comedy</th>
      <td>55383976.0</td>
      <td>30561590.0</td>
      <td>NaN</td>
      <td>111705055.0</td>
    </tr>
    <tr>
      <th>Drama</th>
      <td>NaN</td>
      <td>17146165.5</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Horror</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>68765655.0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Science Fiction &amp; Fantasy</th>
      <td>NaN</td>
      <td>312674899.0</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
  </tbody>
</table>

1) `pd.pivot_table(data=movies, index='genre', columns='rating', values='revenue', aggfunc=np.mean)`

2) `movies.groupby(['genre', 'rating'])['revenue'].mean()`

3) `pd.pivot_table(data=movies, index='rating', columns='genre', values='revenue', aggfunc=np.mean)`

4) `movies.groupby('revenue')[['genre', 'rating']].mean()`


<!--
BEGIN QUESTION
name: q3
-->

In [ ]:
q3_answer = ...

### Question 4: Merging


#### Question 4a

Time to put everything together! Merge `movies` and `count_for_names` to find the number of registered baby names for each director using [`pd.merge`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html?highlight=merge#pandas.merge). Only include names that appear in both `movies` and `count_for_names`.

**Hint:** You might need to convert the `count_for_names` series to a dataframe. Take a look at the ``to_frame`` method of a series to do this. 

Your first row should look something like this:

**Note**: It is ok if you have 2 separate columns with names instead of just one column.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>director</th>
      <th>genre</th>
      <th>movie</th>
      <th>rating</th>
      <th>revenue</th>
      <th>Count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>David</td>
      <td>Action &amp; Adventure</td>
      <td>Deadpool 2</td>
      <td>7</td>
      <td>318344544</td>
      <td>371646</td>
    </tr>
  </tbody>
</table>
</table>


<!--
BEGIN QUESTION
name: q4a
-->

In [ ]:
merged_df = ...
merged_df.head()

#### Question 4b

How many directors in the original `movies` table did not get included in the `merged_df` dataframe? Please hard-code your answer as a number in `q4b`, then explain your answer in 1-2 sentences as a comment below.


<!--
BEGIN QUESTION
name: q4b
-->

In [ ]:
q_4b = ...

# Explain your solution: ...

# Now let's use EEG Data!

This is the first part of a larger project we will be putting together in the next few weeks! We have chosen the [EEG-Alcohol](https://www.kaggle.com/nnair25/Alcoholics) dataset, which contains [EEG (Electroencephalography)](https://en.wikipedia.org/wiki/Electroencephalography) data for two groups - Alcoholic and Control Group. 
![](https://i.imgur.com/ZrmxJRu.jpg)
There are 8 subjects in each group. The 64 electrodes were placed on subject's scalps to measure the electrical activity of the brain. The response values were sampled at 256 Hz (3.9-msec epoch) for 1 second. Each subject was exposed to either a single stimulus (S1) or to two stimuli (S1 and S2) which were pictures of objects chosen from the [1980 Snodgrass and Vanderwart picture set](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.294.1979&rep=rep1&type=pdf). When two stimuli were shown, they were presented in either a matched condition where S1 was identical to S2 or in a non-matched condition where S1 differed from S2.

The purpose of our analysis is going to be to find out if there is a difference in response values for different stimuli between control and alcoholic group. And if so, in what brain regions this happens? 

### Let's do some more imports

In [7]:
import os
import random
from tqdm import tqdm
import matplotlib
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
from scipy.stats import mannwhitneyu

init_notebook_mode(connected=True) ## plotly init
seed = 123
random.seed = seed

In [8]:
print('Total amount of files in SMNI_CMI_TRAIN directory: ' + str(len(os.listdir('../input/SMNI_CMI_TRAIN/'))))

FileNotFoundError: [Errno 2] No such file or directory: '../input/SMNI_CMI_TRAIN/'

In [9]:
filenames_list = os.listdir('../input/SMNI_CMI_TRAIN/') ## list of file names in the directory
EEG_data = pd.DataFrame({}) ## create an empty df that will hold data from each file

for file_name in tqdm(filenames_list):
    temp_df = pd.read_csv('../input/SMNI_CMI_TRAIN/' + file_name) ## read from the file to df
    EEG_data = EEG_data.append(temp_df) ## add the file data to the main df
    
EEG_data = EEG_data.drop(['Unnamed: 0'], axis=1) ## remove the unused column
EEG_data.loc[EEG_data['matching condition'] == 'S2 nomatch,', 'matching condition'] =  'S2 nomatch' ## remove comma sign from stimulus name

FileNotFoundError: [Errno 2] No such file or directory: '../input/SMNI_CMI_TRAIN/'

In [10]:
## here is how the data looks like
EEG_data.head()

NameError: name 'EEG_data' is not defined

## <a id='var_des'>2.2. Variables description</a>

* `trial number`: number of the trial (obviously) 
* `sensor postition`:  position of electrode placed on subject's scalp (based on International 10-20 system)
<img src="https://i.ibb.co/M2GRQNz/670px-International-10-20-system-for-EEG-MCN-svg.png" alt="670px-International-10-20-system-for-EEG-MCN-svg" border="0">
* `sample num`: 0-255
* `sensor value`: value in microvolts 
* `subject identifier`: `a` - Alcoholic; `c` - Control
* `matching condition`: 
 * `S1 obj` - a single object shown; 
 * `S2 match` - object 2 shown in a matching condition (S1 was identical to S2), 
 * `S2 nomatch` - object 2 shown in a non matching condition (S1 differed from S2)
* `channel`: channel number (0-63). Basically, it's the same as `sensor position` column, so one of these columns can be dropped
* `name`: a serial code assigned to each subject
* `time`: inverse of `sample num` measured in seconds


I have changed some `sensor position`s so they match the basics for head topography visualization. Also, I have removed those positions which equal to **X**, **Y** and **ND** since I couldnt figure out which regions they respond to.

In [11]:
## replace some 'sensor position' values
EEG_data.loc[EEG_data['sensor position'] == 'AF1', 'sensor position'] = 'AF3'
EEG_data.loc[EEG_data['sensor position'] == 'AF2', 'sensor position'] = 'AF4'
EEG_data.loc[EEG_data['sensor position'] == 'PO1', 'sensor position'] = 'PO3'
EEG_data.loc[EEG_data['sensor position'] == 'PO2', 'sensor position'] = 'PO4'
## remove rows with undefined positions
EEG_data = EEG_data[(EEG_data['sensor position'] != 'X') & (EEG_data['sensor position'] != 'Y') & (EEG_data['sensor position'] != 'nd')]

NameError: name 'EEG_data' is not defined

## Extra Practice

The best way to learn how to clean data is by doing a bunch of practice!

- Click [this](https://colab.research.google.com/notebooks/mlcc/intro_to_pandas.ipynb) link and complete the quick tutorial.

- Check out this publicly available dataset and figure out how to parse it! Go to http://www.mindbigdata.com/opendb/  and download the MindBigData-MU-v1.0.zip